In [1]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *


## Computer Vision

<h5>1) Vision Learner</h5>

In [2]:
model_meta[resnet50]

{'cut': -2,
 'split': <function fastai.vision.learner._resnet_split(m)>,
 'stats': ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])}

In [3]:
create_head(20,2) #bn_final=True

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): fastai.layers.Flatten(full=False)
  (2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=40, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=False)
)

In [4]:
path = untar_data(URLs.PETS)

KeyboardInterrupt: 

In [ ]:
Path.BASE_PATH = path

In [ ]:
files = get_image_files(path/'images')

In [ ]:
class SiameseImage(fastuple):
    def show(self, ctx=None, **kwargs):
        img1,img2,same_breed = self
        if not isinstance(img1, Tensor):
            if img2.size != img1.size:
                img2 = img2.resize(img.size)
                img1t, img2t = tensor(img1), tensor(img2)
                #we can use einops here
                img1t,img2t = img1t.permute(2,0,1), img2t.permute(2,0,1)
        else: img1t, img2t = img1, img2
        line = img1t.new_zeros(img1t.shape[0], img1t.shape[1], 10)
        return show_image(torch.cat([img1t,line, img2t], dim=2))



In [ ]:
def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

In [ ]:
class SiameseTransform(Transform):
    def __init__(self,files,label_func, splits):
        self.labels = files.map(label_func).unique()
        self.lbl2files = {l: L(f for f in files if label_func(f)==l) for l in self.labels}
        self.label_func = label_func
        self.valid = {f: self._draw(f) for f in files[splits[1]]}
    def encodes(self, f):
        f2,t = self.valid.get(f, self._draw(f))
        img1, img2 = PILImage.create(f), PILImage.create(f2)
        return SiameseImage(img1, img2, t)
    def _draw(self, f):
        same = random.random() < 0.5
        cls = self.label_func(f)
        if not same:
            cls = random.choice(L(l for l in self.labels if l != cls))
        return random.choice(self.lbl2files[cls]), same
        

<h5>Data Splitting and Loading</h5>

In [ ]:
splits = RandomSplitter()(files)
transformed = SiameseTransform(files, label_func, splits)
tls = TfmdLists(files, transformed, splits=splits)
dls = tls.dataloaders(after_item=[Resize(224), ToTensor],
                      after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)])

<h5>Custom Siamese Model Based on Pretrained</h5>

In [ ]:
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder, self.head = encoder, head

    def forward(self, x1, x2):
        ftrs = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)
        return self.head(ftrs)


In [ ]:
encoder = create_body(resnet34, cut=-2)

In [ ]:
head = create_head(512*2, 2, ps=0.5)


In [ ]:
model = SiameseModel(encoder, head)

In [ ]:
def loss_func(out, targ):
    return nn.CrossEntropyLoss()(out, targ.long())

In [ ]:
def siamese_splitter(model): return [params(model.encoder), params(model.head)]

<h5>Constructing Learner</h5>

In [ ]:
learner = Learner(dls, model, loss_func=loss_func, splitter=siamese_splitter, metrics=accuracy)
learner.freeze()

In [ ]:
learner.fit_one_cycle(4, 3e-3)

<h5>Unfreezing and Fine-Tuning</h5>

In [ ]:
learner.unfreeze()
learner.fit_one_cycle(4, slice(1e-6,1e-4)) #low lr for body and high lr for head